# preamble

In [4]:
########## PREAMBLE ###########################################################

# general imports
import openai
import os
import requests
import json

# import scripts
from scripts import transcribe, process_transcript, check_response, prepare_yield_NOTION
from prompts import prompts

# declaring secrets
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
NOTION_API_KEY = os.environ["NOTION_API_KEY"]
NOTION_DATABASE_ID = os.environ["NOTION_DATABASE_ID"]

# declaring static variables
MAX_WHISPER_AUDIO_SIZE = 26262828

MAX_CONTEXT_4K = 4097
MAX_CONTEXT_16K = 4097 * 4

MAX_BLOCK_SIZE = 2000

# declaring dynamic variables
AUDIO_FILE_PATH = "./to_process/230905-reflection_daily.mp3"
THOUGHT = "daily_reflection"

In [7]:
# print all Thought options
for thought in prompts:
    print(thought)

daily_reflection
blog
nederlands


# thought processing

### transcription

In [2]:
print("Transcription started.")
transcript = transcribe(THOUGHT, AUDIO_FILE_PATH) # also deals with long audio files
print("Transcription finished!")

Transcription started.
Transcription finished!


In [3]:
transcript

"This is my daily reflection. I'm going to do it in English to practice, and it's probably easier with Whisper and GPT. Although, I don't know, maybe Dutch is fine as well. So, first about the thought processor idea. It's also a powerful way to reflect. So, thinking out loud and reflecting. That's one thing, sending yourself voice notes, but it's becoming powerful when I actually read them back. When I reflect on the reflection, when I process the reflection. Because often I have ideas, do reflectional insights, so that's good. But if I don't store them somewhere, they're probably going to get lost. So, I need a place to store, a place to share would be nice. Maybe tomorrow I could think of how such a system could look like on different levels. So, a really high meta level, what I was just saying, like a place to reflect or to share thoughts. A place to share things that you would like to either save or process later. So, in a way you can also use it as a to-do list. You can also use i

### transcript processing

In [4]:
print("Text processing started.")
text_response = process_transcript(THOUGHT, transcript)
print("Text processing succes!")

Text processing started.
Text processing succes!


In [5]:
print(type(text_response["choices"][0]["message"]["content"]))
text_response["choices"][0]["message"]["content"]

<class 'str'>


'{"title": "Reflections on the Power of Thought Processing and Personal Development", "summary": "The writer reflects on the idea of thought processing and the importance of storing and reflecting on one\'s thoughts. They also discuss potential ways to create a system for storing and sharing thoughts. In addition, the writer explores their interests in data science and machine learning. They also contemplate the importance of protecting work and sleep, as well as the value of dreaming and pursuing meaningful work. The writer acknowledges their fortunate position regarding money and expresses a desire to create an app and guide on reflection. Finally, they consider the possibility of making their app open source and discuss their aspirations as an entrepreneur.", "lessons": ["The act of reflecting on one\'s thoughts can lead to valuable insights", "Storing and processing thoughts is crucial to prevent them from being lost", "Protecting work and sleep is essential for personal well-being

### check data Notion

In [6]:
content_notion = check_response(text_response)

In [7]:
content_notion

{'title': 'Reflections on the Power of Thought Processing and Personal Development',
 'summary': "The writer reflects on the idea of thought processing and the importance of storing and reflecting on one's thoughts. They also discuss potential ways to create a system for storing and sharing thoughts. In addition, the writer explores their interests in data science and machine learning. They also contemplate the importance of protecting work and sleep, as well as the value of dreaming and pursuing meaningful work. The writer acknowledges their fortunate position regarding money and expresses a desire to create an app and guide on reflection. Finally, they consider the possibility of making their app open source and discuss their aspirations as an entrepreneur.",
 'lessons': "- The act of reflecting on one's thoughts can lead to valuable insights\n- Storing and processing thoughts is crucial to prevent them from being lost\n- Protecting work and sleep is essential for personal well-being

### upload to Notion

In [8]:
url = "https://api.notion.com/v1/pages/"

headers =   {
    "Authorization": f"Bearer {NOTION_API_KEY}",
    "Notion-Version": "2022-06-28",
    "Content-Type": "application/json"
            }

data_database = prepare_yield_NOTION(THOUGHT, NOTION_DATABASE_ID, MAX_BLOCK_SIZE,
                                     content_notion, transcript)

response_get = requests.post(url, headers=headers, json=data_database)
print("Processed text uploading succesful!")

Processed text uploading succesful!


In [ ]:
response_get.json()

---

# Testing

In [13]:
def is_jsonable(x):
    try:
        json.dumps(x)
        return True
    except (TypeError, OverflowError):
        return False

In [40]:
for key in data_database:
    print(is_jsonable(data_database[key]))

True
True
True
True
